In [ ]:
import tensorflow as tf
import numpy as np
from PIL import Image
from architecture import eda_net

# Build a graph

In [ ]:
tf.reset_default_graph()

images = tf.placeholder(tf.uint8, [None, None, None, 3])
logits = eda_net(tf.to_float(images)/255.0, False, 40, 14)
tf.train.init_from_checkpoint('models/run00/model.ckpt-50000', {'/': '/'})
init = tf.global_variables_initializer()

# Predict

In [ ]:
image = Image.open('/mnt/datasets/dan/moda/images/val/102870.jpg')
image

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    L = sess.run(logits, {images: np.expand_dims(np.array(image), 0)}) 
    L = np.argmax(L[0], axis=2)

# Show predicted masks

In [ ]:
color_by_label = {j: np.random.randint(0, 255, size=3, dtype='uint8') for j in range(14)}

def get_color_mask(sparse_mask):
    masks = []
    for j in range(14):
        m = (sparse_mask == j).astype('uint8')
        masks.append(np.expand_dims(m, 2) * color_by_label[j])
        
    return np.stack(masks).max(0)

In [ ]:
image.putalpha(255)

mask = get_color_mask(L)
m = Image.fromarray(mask)
m.putalpha(Image.fromarray(150*(mask > 0).any(2).astype('uint8')))
image.alpha_composite(m)
image